# 语法困境

In [1]:
import nltk
from nltk.grammar import CFG
#从文本生成上下文无关语法 
groucho_grammar = CFG.fromstring('''
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
''')

sent = ['I','shot','an','elephant','in','my','pajamas']
parser = nltk.parse.chart.ChartParser(groucho_grammar)#通用图表分析器
trees = parser.parse(sent)#为语句生成分析树的迭代器
for tree in trees:
    print (tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


# 文法的作用

# 上下文无关文法-简单文法
context-free grammar，CFG

In [2]:
grammar1 = CFG.fromstring("""
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> 'saw' | 'ate' | 'walked'
NP -> 'John' | 'Mary' | 'Bob' | Det N | Det N PP
Det -> 'a' | 'an' | 'the' | 'my'
N -> 'man' | 'dog' | 'cat' | 'telescope' | 'park'
P -> 'in' | 'on' | 'by' | 'with'
""")
sent = ['Mary','saw','Bob']
rd_parser = nltk.parse.recursivedescent.RecursiveDescentParser(grammar1)#, trace=2
for tree in rd_parser.parse(sent):
    print (tree)
    tree.draw()

sent2 = [x.lower() for x in "The dog saw a man in the park".split()]
for tree in rd_parser.parse(sent2):
    print (tree)
    tree.draw()

(S (NP Mary) (VP (V saw) (NP Bob)))
(S
  (NP (Det the) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man) (PP (P in) (NP (Det the) (N park))))))
(S
  (NP (Det the) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man))
    (PP (P in) (NP (Det the) (N park)))))


# 构建文法

In [3]:
grammar1 = nltk.data.load('file:mygrammar.cfg')
sent = "Mary saw Bob".split()
rd_parser = nltk.parse.recursivedescent.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print (tree)
    tree.draw()

(S (NP Mary) (VP (V saw) (NP Bob)))


# 递归下降分析
递归下降分析器在上述过程中建立分析树。带着最初的目标(找到一个 S)，创建 S 根 节点。随着上述过程使用文法的产生式递归扩展，分析树不断向下延伸(故名为递归下降)

In [4]:
rd_parser = nltk.parse.recursivedescent.RecursiveDescentParser(grammar1)
sent ='Mary saw a dog'.split()
for t in rd_parser.parse(sent):
    print (t)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


# 移进-归约分析
一种简单的自下而上分析器是移进-归约分析器。与所有自下而上的分析器一样，移进-归约分析器尝试找到对应文法生产式右侧的词和短语的序列，用左侧的替换它们，直到整个句子归约为一个 S

移进-规约分析器相比递归下降分析器的好处是，它们只建立与输入中的词对应的结构。 此外，每个结构它们只建立一次

In [5]:
nltk.app.srparser_app.app()#用于探索移进-归约解析器的图形工具

In [6]:
sr_parse = nltk.ShiftReduceParser(grammar1)
sent = 'Mary saw a dog'.split()
for t in sr_parse.parse(sent):
    print (t)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


# 符合语句规则的子串表

In [7]:
text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])#返回语法结果:词性->词 rhs仅返回右边具有给定第一项的项目
        wfst[i][i+1] = productions[0].lhs()
    return wfst

def complete_wfst(wfst,tokens,grammar,trace=False):
    index=dict((p.rhs(),p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2,numtokens+1):
        for start in range(numtokens+1-span):
            end=start +span
            for mid in range(start+1,end):
                nt1,nt2 = wfst[start][mid],wfst[mid][end]
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end]= index[(nt1,nt2)]
                    if trace:
                        print ("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % (start,nt1,mid,nt2,end,start,index[(nt1,nt2)],end))
    return wfst

def display(wfst,tokens):
    print ('\nWFST ' + ''.join([("%-4d" % i) for i in range(1, len(wfst))]))
    for i in range(len(wfst)-1):
        print ("%d" % i,end="    ")
        for j in range(1,len(wfst)):
            print ("%-4s" % (wfst[i][j] or '.'),end="")
        print ("")
tokens = "I shot an elephant in my pajamas".split()
wfst0 = init_wfst(tokens, groucho_grammar)
display(wfst0, tokens)
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)
display(wfst1, tokens)
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)


WFST 1   2   3   4   5   6   7   
0    NP  .   .   .   .   .   .   
1    .   V   .   .   .   .   .   
2    .   .   Det .   .   .   .   
3    .   .   .   N   .   .   .   
4    .   .   .   .   P   .   .   
5    .   .   .   .   .   Det .   
6    .   .   .   .   .   .   N   

WFST 1   2   3   4   5   6   7   
0    NP  .   .   S   .   .   S   
1    .   V   .   VP  .   .   VP  
2    .   .   Det NP  .   .   .   
3    .   .   .   N   .   .   .   
4    .   .   .   .   P   .   PP  
5    .   .   .   .   .   Det NP  
6    .   .   .   .   .   .   N   
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]


# 依存关系和依存文法

In [8]:
from nltk.grammar import DependencyGrammar #依赖语法。DependencyGrammar由一组语法组成。每个语法都指定一对词之间的标题/修饰词关系。
from nltk.parse import (
     DependencyGraph,
     ProjectiveDependencyParser,
     NonprojectiveDependencyParser)
groucho_dep_grammar=DependencyGrammar.fromstring("""
'shot' -> 'I' | 'elephant' | 'in'
'elephant' -> 'an' | 'in'
'in' -> 'pajamas'
'pajamas' -> 'my'
""")
print (groucho_dep_grammar)
pdp = ProjectiveDependencyParser(groucho_dep_grammar)
sent = 'I shot an elephant in my pajamas'.split()
trees = pdp.parse(sent)
for tree in trees:
    tree.draw()

Dependency grammar with 7 productions
  'shot' -> 'I'
  'shot' -> 'elephant'
  'shot' -> 'in'
  'elephant' -> 'an'
  'elephant' -> 'in'
  'in' -> 'pajamas'
  'pajamas' -> 'my'


# 树库和文法

In [9]:
from nltk.corpus import treebank
t = treebank.parsed_sents('wsj_0001.mrg')[0]#获取已解析的句子结构
print (t)
def filter(tree):
    child_nodes=[child.label() for child in tree
                if isinstance(child,nltk.Tree)]
    return (tree.label()=='VP') and ('S' in child_nodes)
print ([subtree for tree in treebank.parsed_sents()
 for subtree in tree.subtrees(filter)][0])

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))
(VP
  (VBN named)
  (S
    (NP-SBJ (-NONE- *-1))
    (NP-PRD
      (NP (DT a) (JJ nonexecutive) (NN director))
      (PP
        (IN of)
        (NP (DT this) (JJ British) (JJ industrial) (NN conglomerate))))))


In [10]:
entries = nltk.corpus.ppattach.attachments('training')
table = nltk.defaultdict(lambda: nltk.defaultdict(set))
for entry in entries:
    key = entry.noun1 + '-' + entry.prep + '-' + entry.noun2
    table[key][entry.attachment].add(entry.verb)
    
n=0
for key in sorted(table):
    if len(table[key]) > 1:
        print (key, 'N:', sorted(table[key]['N']), 'V:', sorted(table[key]['V']))
    n+=1
    if n == 100:
        break

nltk.corpus.sinica_treebank.parsed_sents()[3450].draw()

%-below-level N: ['left'] V: ['be']
%-from-year N: ['was'] V: ['declined', 'dropped', 'fell', 'grew', 'increased', 'plunged', 'rose', 'was']
%-in-August N: ['was'] V: ['climbed', 'fell', 'leaping', 'rising', 'rose']
%-in-September N: ['increased'] V: ['climbed', 'declined', 'dropped', 'edged', 'fell', 'grew', 'plunged', 'rose', 'slipped']


# 有害的歧义

In [11]:
grammar=CFG.fromstring("""
S -> NP V NP
NP -> NP Sbar
Sbar -> NP V
NP -> 'fish'
V -> 'fish'
""")
tokens = ["fish"]*5
cp = nltk.ChartParser(grammar)
for tree in cp.parse(tokens):
    print (tree)

(S (NP fish) (V fish) (NP (NP fish) (Sbar (NP fish) (V fish))))
(S (NP (NP fish) (Sbar (NP fish) (V fish))) (V fish) (NP fish))


# 加权文法

In [17]:
def give(t):
    return t.label() == 'VP' and len(t)>2 and t[1].label() == 'NP' \
        and (t[2].label() == 'PP-DTV' or t[2].label() == 'NP') \
        and ('give' in t[0].leaves() or 'gave' in t[0].leaves())

def sent(t):
    return ' '.join(token for token in t.leaves() if token[0] not in '*-0')

def print_node(t,width):
    output = "%s %s: %s / %s: %s" %\
        (sent(t[0]),t[1].label(),sent(t[1]),t[2].label(),sent(t[2]))
    if len(output) > width:
        output = output[:width] + "..."
    print (output)
    
for tree in nltk.corpus.treebank.parsed_sents():
    for t in tree.subtrees(give):
        print_node(t,72)

gave NP: the chefs / NP: a standing ovation
give NP: advertisers / NP: discounts for maintaining or increasing ad sp...
give NP: it / PP-DTV: to the politicians
gave NP: them / NP: similar help
give NP: them / NP: 
give NP: only French history questions / PP-DTV: to students in a Europe...
give NP: federal judges / NP: a raise
give NP: consumers / NP: the straight scoop on the U.S. waste crisis
gave NP: Mitsui / NP: access to a high-tech medical product
give NP: Mitsubishi / NP: a window on the U.S. glass industry
give NP: much thought / PP-DTV: to the rates she was receiving , nor to ...
give NP: your Foster Savings Institution / NP: the gift of hope and free...
give NP: market operators / NP: the authority to suspend trading in futu...
gave NP: quick approval / PP-DTV: to $ 3.18 billion in supplemental appr...
give NP: the Transportation Department / NP: up to 50 days to review any...
give NP: the president / NP: such power
give NP: me / NP: the heebie-jeebies
give NP: holders / NP: 

In [21]:
from nltk.grammar import PCFG
grammar = PCFG.fromstring("""
S -> NP VP [1.0]
VP -> TV NP [0.4]
VP -> IV [0.3]
VP -> DatV NP NP [0.3]
TV -> 'saw' [1.0]
IV -> 'ate' [1.0]
DatV -> 'gave' [1.0]
NP -> 'telescopes' [0.8]
NP -> 'Jack' [0.2]
""")
print (grammar)
viterbi_parser = nltk.parse.viterbi.ViterbiParser(grammar)#一个自下而上的pcfg解析器
for vp in viterbi_parser.parse(['Jack','saw','telescopes']):
    print (vp)

Grammar with 9 productions (start state = S)
    S -> NP VP [1.0]
    VP -> TV NP [0.4]
    VP -> IV [0.3]
    VP -> DatV NP NP [0.3]
    TV -> 'saw' [1.0]
    IV -> 'ate' [1.0]
    DatV -> 'gave' [1.0]
    NP -> 'telescopes' [0.8]
    NP -> 'Jack' [0.2]
(S (NP Jack) (VP (TV saw) (NP telescopes))) (p=0.064)
